In [1]:
!which python
!python -V

/Users/ivustianiu/Library/Caches/pypoetry/virtualenvs/mlops-zoomcamp-0GqB-Z-k-py3.10/bin/python
Python 3.10.13


In [2]:
import pandas as pd
import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import root_mean_squared_error

In [3]:
df = pd.read_parquet('../../data/yellow_tripdata_2023-01.parquet')
print(df.shape)

(3066766, 19)


In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)
print('standard deviation:', df['duration'].std())

standard deviation: 42.59435124195458


In [5]:
n_rows_before = len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]
n_rows_after = len(df)
print('% of rows left:', n_rows_after / n_rows_before * 100)

% of rows left: 98.1220282212598


In [6]:
categorical = ['PULocationID', 'DOLocationID']

df[categorical] = df[categorical].astype(str)
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
print(X_train.shape)

(3009173, 515)


In [7]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
print(root_mean_squared_error(y_train, y_pred))

7.649261927686161


In [8]:
def read_dataframe(filename, remove_outliers: bool = True):
    df = pd.read_parquet(filename)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda td: td.total_seconds() / 60)
    if remove_outliers:
        df = df[(df.duration >= 1) & (df.duration <= 60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [9]:
df_train = read_dataframe('../../data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('../../data/yellow_tripdata_2023-02.parquet')

In [10]:
len(df_train), len(df_val)

(3009173, 2855951)

In [11]:
categorical = ['PULocationID', 'DOLocationID']
dv = DictVectorizer()
train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.811817957524739

In [14]:
with open('../../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Q1: 19 <br>
Q2: 42.59 <br>
Q3: 98% <br>
Q4: 515 <br>
Q5: 7.64 <br>
Q6: 7.81 <br>